# **Notebook 6: Final Pipeline and Deployment Preparation**

## Objectives

The objective of this notebook is to consolidate the final machine learning pipeline and prepare the trained model(s) for deployment. This includes serializing the pipeline, saving outputs, and creating essential documentation for integration into the deployment environment.

## Inputs

* **Trained Models and Hyperparameters**
  * Best-performing models from the model training and evaluation notebook, including their hyperparameters.
* **Processed Dataset**
  * Cleaned and feature-engineered datasets ready for input into the pipeline.
* **Evaluation Metrics and Feature Importances**
  * Outputs from the model training notebook to inform pipeline structure and deployment requirements.

## Outputs

* **Serialized Final Pipeline**
  * The complete pipeline, including preprocessing and the best-performing model, saved for deployment.
* **Deployment-Ready Artifacts**
  * Files required for model integration, such as serialized objects and configuration files.
* **Documentation**
  * Summary of the pipeline, deployment steps, and integration instructions.

## Additional Comments

* This ntoebook serves as the final step before integrating the pipeline into the deployment environment.
* The pipeline will include preprocessing steps, feature selection, and the chosen model for prediction.
* Key deployment considerations, such as scalability and maintainability, will be addressed.


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

## Requirements

### Import Libraries

### Verify Dependancies

### Load Saved Artifacts

---

## Pipeline Design

### Preprocessing Pipeline

### Model Intergration

### Prediction Pipeline

---

## Validation

### Test Pipeline

### Output Verification

---

## Serialization

### Save Pipeline

### Save Other Artifacts

---

## Deployment Preparation

### Folder Structure

### Inference Example

### Environment File

---

## Documentation

### Detailed Steps

### Usage Notes

---

## Future Maintenance

### Recommendations